In [1]:
import pickle
import librosa
import numpy as np
import wave
import pandas as pd
import sounddevice as sd



In [3]:
with open('random_search_RF.pickle', 'rb') as file:
    seu_modelo = pickle.load(file)

In [83]:
# Configurações de áudio
fs = 44100  # Frequência de amostragem
duration = 5  # Duração da gravação em segundos

output_file = 'teste.wav'

# Captura de áudio em tempo real
audio_data = sd.rec(int(fs * duration), samplerate=fs, channels=1, dtype='int16')
sd.wait()
audio_data2 = audio_data.astype(np.int16)

with wave.open(output_file, 'wb') as wf:
    wf.setnchannels(1)
    wf.setsampwidth(2)
    wf.setframerate(fs)
    wf.writeframes(audio_data2.tobytes())

In [84]:
def feature_extract(file):
    """
    Define function that takes in a file an returns features in an array
    """
    
    #get wave representation
    y, sr = librosa.load(file)
        
    #determine if instruemnt is harmonic or percussive by comparing means
    y_harmonic, y_percussive = librosa.effects.hpss(y)
    if np.mean(y_harmonic)>np.mean(y_percussive):
        harmonic=1
    else:
        harmonic=0
        
    #Mel-frequency cepstral coefficients (MFCCs)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    #temporal averaging
    mfcc=np.mean(mfcc,axis=1)
    
    #get the mel-scaled spectrogram
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128,fmax=8000)  
    #temporally average spectrogram
    spectrogram = np.mean(spectrogram, axis = 1)
    
    #compute chroma energy
    chroma = librosa.feature.chroma_cens(y=y, sr=sr)
    #temporally average chroma
    chroma = np.mean(chroma, axis = 1)
    
    #compute spectral contrast
    contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    contrast = np.mean(contrast, axis= 1)
    
    return [harmonic, mfcc, spectrogram, chroma, contrast]

In [85]:
#create dictionary to store all test features
dict_test = {}

#extract the features
features = feature_extract('teste.wav')

#add dictionary entry
dict_test[file] = features



In [86]:
#convert dict to dataframe
features_test = pd.DataFrame.from_dict(dict_test, orient='index',
                                       columns=['harmonic', 'mfcc', 'spectro', 'chroma', 'contrast'])

features_test.head()

,harmonic,mfcc,spectro,chroma,contrast
<_io.BufferedReader name='random_search_RF.pickle'>,0,"[-457.4784, 166.36539, 39.025726, 23.273085, 1...","[0.036309533, 0.13584355, 0.13060197, 0.329707...","[0.16214408, 0.16859517, 0.5722397, 0.16206378...","[19.019316126907423, 16.59826979766367, 19.210..."


In [87]:
#extract mfccs
mfcc_test = pd.DataFrame(features_test.mfcc.values.tolist(),index=features_test.index)
mfcc_test = mfcc_test.add_prefix('mfcc_')

#extract spectro
spectro_test = pd.DataFrame(features_test.spectro.values.tolist(),index=features_test.index)
spectro_test = spectro_test.add_prefix('spectro_')


#extract chroma
chroma_test = pd.DataFrame(features_test.chroma.values.tolist(),index=features_test.index)
chroma_test = chroma_test.add_prefix('chroma_')


#extract contrast
contrast_test = pd.DataFrame(features_test.contrast.values.tolist(),index=features_test.index)
contrast_test = chroma_test.add_prefix('contrast_')

#drop the old columns
features_test = features_test.drop(labels=['mfcc', 'spectro', 'chroma', 'contrast'], axis=1)

#concatenate
df_features_test=pd.concat([features_test, mfcc_test, spectro_test, chroma_test, contrast_test],
                           axis=1, join='inner')
df_features_test.head()

,harmonic,mfcc_0,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,...,contrast_chroma_2,contrast_chroma_3,contrast_chroma_4,contrast_chroma_5,contrast_chroma_6,contrast_chroma_7,contrast_chroma_8,contrast_chroma_9,contrast_chroma_10,contrast_chroma_11
<_io.BufferedReader name='random_search_RF.pickle'>,0,-457.478394,166.365387,39.025726,23.273085,14.992258,19.828815,9.292751,0.671396,-1.564252,...,0.57224,0.162064,0.106824,0.06865,0.204034,0.111953,0.102622,0.487109,0.238684,0.182297


In [88]:
targets_test = []

targets_test.append('test')
df_features_test['targets'] = targets_test
X_test = df_features_test.drop(labels=['targets'], axis=1)

In [89]:
result = seu_modelo.predict(X_test)
if(result == 0):
    print("Bass")
elif(result == 1):
    print("Brass")
elif(result == 2):
    print("Flute")
elif(result == 3):
    print("Guitar")
elif(result == 4):
    print("Keyboard")
elif(result == 5):
    print("Mallet")
elif(result == 6):
    print("Organ")
elif(result == 7):
    print("Reed")
elif(result == 8):
    print("String")
elif(result == 9):
    print("Synth Lead")
elif(result == 10):
    print("Vocal")
else:
    print("Erro")


print(result)

String
[8]
